In [1]:
!pip install git+https://github.com/suessmann/pytorch-fm.git

  Cloning https://github.com/suessmann/pytorch-fm.git to /tmp/pip-req-build-yq5k2bxa
  Running command git clone --filter=blob:none --quiet https://github.com/suessmann/pytorch-fm.git /tmp/pip-req-build-yq5k2bxa
  Resolved https://github.com/suessmann/pytorch-fm.git to commit 2ab8cf3538f974a438ceb6008567885b9129879f
  Preparing metadata (setup.py) ... done


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchfm.model.ffm import FieldAwareFactorizationMachineModel

from tqdm.notebook import tqdm
from sklearn.metrics import log_loss, roc_auc_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


В целом в части тренировки модели изменений нет, просто проведем дополнительную предобработку значений g_0 и g_1

In [3]:
df = pd.read_csv('/kaggle/input/hw4-recsys/data (1).csv')
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


In [4]:
df.shape

(15821472, 17)

In [5]:
df.drop(["rate0", "rate1", "campaign_clicks"], axis=1, inplace=True)

Удалим отрицательные значения в g_0 и g_1, ведь это дисперсия

In [6]:
df = df[(df.g0 >= 0) & (df.g1 >= 0)]

Проведём предобработку

In [7]:
categorical_features = ["zone_id", "os_id", "country_id"]

In [8]:
for col in categorical_features:
    lt = np.quantile(np.unique(df[col], return_counts=True)[1], 0.999)
    df.loc[df[col].value_counts()[df[col]].values < lt, col] = df[col].unique().max() + 1

In [9]:
len(df['zone_id'].unique())

5

Посчитаем филды и приведём в порядок индексы

In [10]:
all_categorical_features = ["zone_id", "banner_id", "os_id", "country_id", "oaid_hash"]

for field in all_categorical_features:
    uniques = np.unique(df[field])
    mapping = {v: i for i, v in enumerate(uniques)}
    df[field] = df[field].map(mapping)

In [11]:
field_dim = [np.max(df[k]) + 1 for k in ["zone_id", "banner_id", 
                                         "oaid_hash", "os_id", 
                                         "country_id", "impressions"]]
field_dim

[5, 1618, 6504954, 2, 2, 2]

Теперь отделим тестовые данные от трейна. Для теста возьмём последний доступный из данных день

In [12]:
pd.to_datetime(df.date_time)

0          2021-09-27 00:01:30
1          2021-09-26 22:54:49
2          2021-09-26 23:57:20
3          2021-09-27 00:04:30
4          2021-09-27 00:06:21
                   ...        
15821467   2021-10-02 15:51:35
15821468   2021-09-27 22:03:14
15821469   2021-10-02 17:41:10
15821470   2021-09-29 00:39:32
15821471   2021-09-28 07:00:18
Name: date_time, Length: 15801721, dtype: datetime64[ns]

In [13]:
df.date_time = pd.DatetimeIndex(pd.to_datetime(df.date_time)).normalize()

In [14]:
df.date_time.max()

Timestamp('2021-10-02 00:00:00')

In [15]:
df[df.date_time == '2021-10-02']

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,impressions,clicks
164,2021-10-02,4,74,5646166,1,1,76,0.055551,-2.926980,401,0.030272,-3.390642,1,1
166,2021-10-02,2,45,4784995,1,0,46,0.017521,-1.377320,11464251,0.085038,-3.329596,1,1
168,2021-10-02,4,74,2875758,0,1,76,0.171074,-3.112081,11464252,0.079034,-1.907685,1,1
169,2021-10-02,2,45,3650682,1,0,46,0.017439,-2.493974,0,0.017624,-3.889516,1,1
359,2021-10-02,2,2,3412826,1,0,2,0.020414,-2.154111,49,0.068041,-3.088063,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15821452,2021-10-02,4,87,5674215,1,1,89,0.043305,-2.983741,7,0.014551,-3.074290,1,0
15821455,2021-10-02,4,130,287076,0,0,132,0.031065,-3.745272,14623598,0.099036,-2.287568,1,0
15821461,2021-10-02,3,51,2983271,0,1,52,0.008300,-3.538011,14623601,0.028516,-2.011854,1,0
15821467,2021-10-02,4,528,3050713,0,1,530,0.040037,-3.495224,481,0.039634,-3.493091,1,0


In [16]:
# 'g0', 'g1', "banner_id0", "banner_id1"]
test_data = df[df.date_time == '2021-10-02'].drop('date_time', axis=1)
train_data = df[df.date_time < '2021-10-02'].drop('date_time', axis=1)

test_pi0 = test_data.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id1'],axis=1).values
test_pi1 = test_data.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id'],axis=1)
test_pi1['banner_id'] = test_pi1['banner_id1']
test_pi1 = test_pi1.drop('banner_id1', axis=1).values

aux_params = test_data[['coeff_sum0', 'coeff_sum1', 'g0', 'g1']]

train_data = train_data.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id1'],axis=1)
test_data = test_data.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id1'],axis=1)

test_data.shape, train_data.shape

((2124072, 7), (13677649, 7))

Создадим всё что нужно для торча

In [18]:
class ClicksDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        clck = self.data[idx, -1]
        feats = self.data[idx, :-1]
        return feats, clck

In [19]:
train_dataset = ClicksDataset(train_data.values)
test_dataset = ClicksDataset(test_data.values)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)

In [21]:
model = FieldAwareFactorizationMachineModel(field_dim, 5).cuda()
optim = torch.optim.Adam(model.parameters(), lr=3e-3)
criterion = torch.nn.BCELoss()

In [22]:
pbar = tqdm(total=len(train_loader))
for feats, clck in train_loader:
    feats = torch.LongTensor(feats).to('cuda')
    clck = clck.float().to('cuda')
    pred = model(feats)
    loss = criterion(pred, clck)
    loss.backward()
    optim.step()
    optim.zero_grad()
    pbar.set_description(f"Training loss: {loss.cpu().detach().item()}")
    pbar.update(1)

  0%|          | 0/13358 [00:00<?, ?it/s]

In [23]:
test_dataset = ClicksDataset(test_data.values)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)

In [24]:
y_true = []
y_pred = []

for feats, clck in tqdm(test_loader):
    feats = torch.LongTensor(feats).to('cuda')
    y_true += clck.tolist()
    pred = model(feats)
    y_pred += pred.detach().cpu().tolist()
    
    assert len(y_true) == len(y_pred), f'{pred.shape}, {clck.shape}'

  0%|          | 0/2075 [00:00<?, ?it/s]

Скоры на тесте

In [25]:
print("Log loss", log_loss(y_true, y_pred))
print("ROC AUC", roc_auc_score(y_true, y_pred))

Log loss 0.14829285758256933
ROC AUC 0.7405985074006338


Посчитаем нашей моделью новые коэффициенты для баннеров

In [26]:
test_pi0_dataset = ClicksDataset(test_pi0)
test_pi0_loader = DataLoader(test_pi0_dataset, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)

test_pi1_dataset = ClicksDataset(test_pi1)
test_pi1_loader = DataLoader(test_pi1_dataset, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)

In [27]:
coeff_sum0_model = []
coeff_sum1_model = []

for feats, clck in tqdm(test_pi0_loader):
    feats = torch.LongTensor(feats).to('cuda')
    logits = model(feats)
    coeff_sum0_model += logits.detach().cpu().tolist()

  0%|          | 0/2075 [00:00<?, ?it/s]

In [28]:
for feats, clck in tqdm(test_pi1_loader):
    feats = torch.LongTensor(feats).to('cuda')
    logits = model(feats)
    coeff_sum1_model += logits.detach().cpu().tolist()

  0%|          | 0/2075 [00:00<?, ?it/s]

In [29]:
from scipy.stats import norm

In [30]:
coeff_sum0_model = np.array(coeff_sum0_model)
coeff_sum1_model = np.array(coeff_sum1_model)

Вероятность, что одна сл. вел. больше другой:

$X \sim N(\mu_1, \sigma^2_1)$

$Y \sim N(\mu_2, \sigma^2_2)$

$P(Y<X) = P(Y - X < 0) = F_{Y-X}(0)$

$Y - X \sim N(\mu_2 - \mu_1, \sqrt{\sigma^2_2 + \sigma^2_1})$

In [31]:
pi0 = norm.cdf(0, loc=aux_params['coeff_sum1'] - aux_params['coeff_sum0'],
              scale=np.sqrt(aux_params['g0'] ** 2 + aux_params['g1'] ** 2) + 1e-10)

pi1 = norm.cdf(0, loc=coeff_sum1_model - coeff_sum0_model,
              scale=np.sqrt(aux_params['g0'] ** 2 + aux_params['g1'] ** 2) + 1e-10)

In [32]:
reward = test_data['clicks'].values
cips = np.mean(reward * np.minimum(pi1 / (pi0 + 1e-10), 10))
print(f"clipped ips: {cips}")

clipped ips: 0.04702011658615194
